In [6]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    # Tools準備
    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    tool_node = ToolNode(tools)

    # LLM準備
    llm = ChatOpenAI(model_name=model_name)
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノード準備
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}
    
    # グラフ準備
    graph_builder = StateGraph(State)
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", tool_node)
    
    graph_builder.add_conditional_edges(
         "chatbot",
         tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    graph_builder.set_entry_point("chatbot")

    # メモリ設定
    memory = MemorySaver()
    compiled_graph = graph_builder.compile(checkpointer=memory)

    return compiled_graph

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode = "values"
    )

    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました！")
        break
    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？
1たす2は？
1たす2は3です。何か他にお手伝いできることはありますか？
台湾観光について検索結果を教えて

[{"url": "https://www.knt.co.jp/travelguide/kaigai/027/", "content": "国内\n\n海外\n\n台湾・台北観光のおすすめスポット19選！\n\n台湾・台北のツアーを日付・条件から探す\n\n台湾は日本から直行便で約4時間ほどで行ける人気の観光地。週末や連休を利用して気軽に行ける台湾は、グルメや観光、写真映えスポット、ショッピングなど幅広く楽しむことができます。国立故宮博物院や中正紀念堂など台湾の歴史あふれるエリア、若者に人気のショッピングエリアをはじめ、台北から少し足を伸ばせば、ノスタルジックな雰囲気と海を一望できる風光明媚な街並みが広がる九份があります。今回は台北を訪れたら絶対にチェックしておきたいスポットをご紹介します。\n\n目次\n\n【台北定番の観光スポット】\n\n高さ約509ｍを誇る台湾の街のシンボルタワー「台北101」\n\n世界四大博物館の1つ「国立故宮博物院」\n\n台北市で最大規模を誇る「士林観光夜市」\n\n美食がそろう夜市「饒河街観光夜市」\n\n台湾の歴史を象徴する場所「国立中正紀念堂」\n\nレトロな街並みが残る「迪化街」\n\n台湾のベニスと呼ばれる港町「淡水」\n\n商売繁盛の神様をまつる伝統的な廟「行天宮」\n\n英霊をまつる神聖な場所「忠烈祠」\n\n名店がずらりと並ぶ「永康街」 [...] いかがでしたでしょうか。何年もの歴史を受け継いで今なお残る寺院や、毎週人々が集まる写真に映える観光スポットなど、台北にはたくさんの観光名所がございます。この記事を参考に、施設、予算などを比較しながら、旅の目的にあった観光をお楽しみください。\n\n※新型コロナウイルス感染拡大に伴い、一部施設の利用中止や営業時間の変更、ご提供内容の変更が発生する場合があります。※記載の情報は随時更新を行なっておりますが、予告なく変更となる場合がございます。あくまでも観光の参考としてご覧ください。※ご利用には別途代金のかかるサービスもございます。現地にてご確認ください。\n\n※2022年11月1日時点の情報です。※最新